In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  9 20:04:02 2019

@author: vatsa
"""

#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv
import string
import pandas as pd
import numpy as np
import re

#to preprocess tweets
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 


from textblob import TextBlob
import string


#Twitter API credentials
####input your credentials here
consumer_key = 'XXXXXXXX'
consumer_secret = 'XXXXXXXXX'
access_key = 'XXXXXXXXX'
access_secret = 'XXXXXXXXX'



Function to extract the tweets

In [ ]:

def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []	
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.search(q = screen_name, rpp=100, count=1000, tweet_mode='extended')
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[tweet.id_str, tweet.created_at, tweet.full_text.encode("utf-8") if tweet.full_text.startswith("RT @") == False else tweet.retweeted_status.full_text.encode("utf-8")] for tweet in alltweets]
    
    #write the csv	
    with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    
    pass

In [ ]:
#pass in the username of the account you want the tweets of
get_all_tweets("@dominos_india")

Developing the dataframe out of the tweets extracted

In [ ]:
combi = pd.read_csv("C:/Users/vatsa/@dominos_india_tweets.csv")

# remove special characters, numbers, punctuations
combi['tidy_tweet'] = combi['text'].str.replace("[^a-zA-Z#]", " ")

# remove twitter handles (@user)
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tidy_tweet'], "@[\w]*")



Incorporating the emoticons set for better sentiment generation

In [ ]:

emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
    

In [ ]:
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
    

In [ ]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
 

In [ ]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)


Function to clean the tweets

In [ ]:

def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(tweet)
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub('http\S+', '', tweet) #remove links
    tweet = re.sub('RT', '', tweet) #remove RT of retweet
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)

Performing sentiment analysis using textBlob and seperting out the tweets related to a given product

In [ ]:

combi["ptweets"] = combi['tidy_tweet'].apply(lambda x : clean_tweets(x))

combi["ptweets"].head()

combi["polarity"] = combi['ptweets'].apply(lambda x : TextBlob(x).sentiment.polarity)

combi["subjectivity"] = combi['ptweets'].apply(lambda x : TextBlob(x).sentiment.subjectivity)

combi.head()

pizza= []

combi['pizza'] = combi['ptweets'].apply(lambda x: x if "pizza" in x else '')
combi.to_csv('combi.csv')